In [1]:
import os
import time

import samlab.dashboard

from selenium.webdriver import ActionChains, Firefox
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait as Wait
from selenium.webdriver.support.expected_conditions import presence_of_element_located as element_located

dashboard = samlab.dashboard.Server(config=False, quiet=True)
dashboard.ready(timeout=10)
print(dashboard.uri)

options = Options()
#options.headless = True
options.add_argument("--width=1024")
options.add_argument("--height=768")

browser = Firefox(options=options)
actions = ActionChains(browser)

os.makedirs("dashboard", exist_ok=True)

http://127.0.0.1:64417


In [2]:
def screenshot(path, delay=0.5):
    path = os.path.join(os.getcwd(), "dashboard", path)
    print(f"Saving screenshot {path}")
    time.sleep(delay)
    browser.save_screenshot(os.path.join(os.getcwd(), path))
    time.sleep(delay)


def toggle_datasets_menu():
    browser.find_element_by_css_selector("#samlab-dashboard-datasets-menu button").click()


def toggle_favorites_menu():
    browser.find_element_by_css_selector("#samlab-dashboard-favorites-menu button").click()

    
def toggle_operations_menu():
    browser.find_element_by_css_selector("#samlab-dashboard-operations-menu button").click()
    
    
def widgets(*, component=None, title=None, xpath=None):
    elements = browser.find_elements_by_xpath("//div[contains(concat(' ',normalize-space(@class),' '),' grid-stack-item ')]")
    
    if component is not None:
        elements = [element for element in elements if element.find_elements_by_xpath(f".[contains(concat(' ',normalize-space(@class),' '), ' {component} ')]")]
    
    if title is not None:
        elements = [element for element in elements if element.find_elements_by_xpath(f"//h1[text()='{title}']")]
    
    if xpath is not None:
        elements = [element for element in elements if element.find_elements_by_xpath(xpath)]

    return elements
                    

def widget(*, component=None, title=None, xpath=None):
    elements = widgets(component=component, title=title, xpath=xpath)
                    
    if not elements:
        raise ValueError("No matching widget.")
    if len(elements) > 1:
        raise ValueError("More than one matching widget.")
        
    return elements[0]

In [3]:
# Default home page.
browser.get(dashboard.uri)
Wait(browser, timeout=10).until(element_located((By.ID, "samlab-dashboard-favorites-menu")))
screenshot("welcome.png")

Saving screenshot /Users/tshead/src/samlab/docs/dashboard/welcome.png


In [4]:
# Show the favorites menu.
toggle_favorites_menu()
screenshot("empty-favorites-menu.png")
toggle_favorites_menu()

# Show the datasets menu.
toggle_datasets_menu()
screenshot("empty-datasets-menu.png")
toggle_datasets_menu()

# Show the operations menu.
toggle_operations_menu()
screenshot("operations-menu.png")

Saving screenshot /Users/tshead/src/samlab/docs/dashboard/empty-favorites-menu.png
Saving screenshot /Users/tshead/src/samlab/docs/dashboard/empty-datasets-menu.png
Saving screenshot /Users/tshead/src/samlab/docs/dashboard/operations-menu.png


In [5]:
# Display the About menu item.
element = browser.find_element_by_link_text("About")
ActionChains(browser).move_to_element(element).perform()
#time.sleep(0.5)
screenshot("before-about.png")

# Open an About widget.
element.click()
about_widget = widget(component="samlab-markup-viewer-widget", title="About Samlab")
screenshot("after-about.png")

# Hover over the About widget.
actions.move_to_element(about_widget).perform()
screenshot("hover-about.png")

# Hover over the About widget drag region.
element = about_widget.find_element_by_css_selector(".grid-stack-item-handle")
actions.move_to_element(element).perform()
screenshot("before-drag.png")

Saving screenshot /Users/tshead/src/samlab/docs/dashboard/before-about.png
Saving screenshot /Users/tshead/src/samlab/docs/dashboard/after-about.png
Saving screenshot /Users/tshead/src/samlab/docs/dashboard/hover-about.png
Saving screenshot /Users/tshead/src/samlab/docs/dashboard/before-drag.png


In [6]:
# Drag the About widget to a new location.
element = about_widget.find_element_by_css_selector(".grid-stack-item-handle")
actions.move_to_element(about_widget).perform()

# actions.drag_and_drop_by_offset(element, 500, 0).perform()
# time.sleep(0.5)
# screenshot("after-drag.png")

In [7]:
browser.quit()
dashboard.stop()

For this tutorial, let's begin by starting the server from the command line:

`$ samlab-dashboard`

The server will start, and automatically open the dashboard interface in a web browser:

<img class="browser" src="dashboard/welcome.png">

Let's examine the menus in order. The Favorites menu lists dashboard layouts that have been favorited for later recall or sharing with collaborators.  Since we haven't favorited anything yet, the menu is empty:

<img class="browser" src="dashboard/empty-favorites-menu.png">

The Datasets menu lists all of the data sources that have been registered with the server. Since we haven't registered any yet, it too is empty: 

<img class="browser" src="dashboard/empty-datasets-menu.png">

The Operations menu provides a set of generic commands for managing the dashboard layout, information lists, and displaying acknowledgements:

<img class="browser" src="dashboard/operations-menu.png">

Let's use the latter to get some familiarity with the Samlab user interface.  Start by clicking on the *Operations > About* menu:

<img class="browser" src="dashboard/before-about.png">

This will open an *About Widget*:

<img class="browser" src="dashboard/after-about.png">

Samlab *widgets* are windows that can be moved and resized to arrange the dashboard however you like.  If you hover the mouse over a widget, you will see that it has a *close button* in the upper-right corner, and a *resize handle* in the lower-right corner:

<img class="browser" src="dashboard/hover-about.png">

To reposition a widget, hover the mouse over the *move region* near the top of the widget:

<img class="browser" src="dashboard/before-drag.png">

Then drag the widget to a new location:

<img class="browser" src="dashboard/after-drag.png">

